# 1.1 Introduction

The objective of this notebook is to get you started in Julia with a real example: the Gradient Descent algorithm. We have chosen this algorithm because:

* **The mathematical problem is simple and ubiquitous**: given a function, find a minimum. 
* **The algorithm formulation is simple**: just take small leaps following the steepest path down.
* **The algorithm admits plenty of variations**: we can play with several variations to explore the Julia lenguage in greater depth 


# 1.2 Mathematical formulation

*Gradient descent* is one of the simplest methods for finding optima. As [Wikipedia](https://en.wikipedia.org/wiki/Gradient_descent) puts it, its core idea is that a differentiable function $f(x)$ decreases fastest if one follows the direction of  $− \nabla f$ (the direction of greatest slope). Since this is in general only valid close to our starting point, the gradient descent corresponds to the following sequence:

$$ x_{n+1} = x_n - \alpha \nabla f(x_n) $$

where $\alpha$ is some parameter chosen in such a way that we keep decreasing $f$ in each step (or at least we try so). If $\alpha$ is too high we could land even farther from the minimum in the next iterate! The resulting sequence $x_n$ will look something like this:

![gradient descent gif](gradient_descent.gif)

(By the way, we will also learn to generate animations like this. Yay!)



# 1.3 The pseudo-code

We then want some code that given a function `f`, its derivative or gradient `Df` and a starting point `x`, returns a minimum and the value of `f` at this minimum. The code should look something like this:

```julia
function gradient_descent(f, Df, x; alpha = 0.1)
    while (x is not close enough to a minimum)
        x = x - alpha*Df(x)
    end
    return x, f(x)
end
```

If you are not familiar with the algorithm you may visit [Wikipedia's page](https://en.wikipedia.org/wiki/Gradient_descent#Computational_examples) and check the code in your favorite programming language.

Let's learn little by little all we need to code such function!

# 1.4 Variables, operations and functions

Variable declaration and arithmetic operations won't look challenging.

In [24]:
x = 1
s = "Hello world!"
tup = (x, s) # This is a tuple, a collection of (possible distinct) elements

println(x)   # The function print doesn't add a new line after
println(s)
println(tup)

1
Hello world!
(1, "Hello world!")


In [26]:
y = x + 2
z = y * 5
z += 1 # inplace adition

print("The final value of z is $z") # The dollar is used for formatting variables into a string

The final value of z is 16

The standard way of defining a function begins with the `function` keyword and must end with an `end`. They return the last computed expression (or whatever follows a `return`):

In [3]:
function f(x)
    x^2
end

f(-2)

4

In [4]:
# But sometimes it's simpler to just do:
Df(x) = 2*x

Df(1)

2

# 1.5 Loops

`for` loops will be familiar for **Matlab** users:

In [5]:
for i=1:5
    print(i," ")
end

1 2 3 4 5 

Anyway, **Python** users will also feel *almost* at home:

In [6]:
for i in 1:5
    print(i," ")
end

1 2 3 4 5 

Of course there are also while-loops:

In [7]:
i = 0
while i < 2
    print(i, " ")
    i+=1
end

0 1 

And `if` statements don't bring many surprises:

In [8]:
a = 0
if a < 0
    print("a is negative")
elseif a > 0
    print("a is positive")
else
    print("a is zero")
end

a is zero

<div class="alert alert-info">
    <h3>A quick but important warning about scopes</h3>
    <p>When it comes to defining and accessing variables, Julia has a <i>global scope</i> (the basic level of a Jupyter Notebook cell, the REPL,...) and <i>local scopes</i> (a function, a `for` loop...). A variable that is declared inside a local scope cannot be accesed from outside; this behavior is pretty common in other languages when it comes to functions, <b>but</b> in Julia this also extends to loops, which at first can be kind of weird. 
</p>
    <p> Check the following code and try to fix it:  </p>
</div>


In [9]:
for i in 1:1
    b = 1
end
b

UndefVarError: UndefVarError: b not defined

However, `if` statements don't introduce any new scope:

In [10]:
if true
    c = 1
end
c

1

# 1.6 A first version of the gradient descent

With all these tools we are ready to code a preliminary version of the gradient descend algorithm:

In [11]:
function gradient_descent(f, Df, x)
    alpha = 0.1
    
    for i in 1:100
        x = x - alpha*Df(x)
    end
    
    return (x, f(x))
end

gradient_descent (generic function with 1 method)

Notice that right now we have a fixed number of iterations, and the learning rate is fixed to `alpha = 0.1`. However, it kind of does the job; for `f` and `Df` as defined above it yields:

In [12]:
x_opt, f_opt = gradient_descent(f, Df, 1) 

(2.0370359763344877e-10, 4.1495155688809995e-20)

Since the real values for `x_opt` and `f_opt` are 0, the result is not too bad. 

We would like now to have a bit more flexibility and robustness. Let us include `alpha` as an *optional argument*, and also include a *docstring* before the function; this is, some help string that tells us how the algorithm works.

In [13]:
""" Second version of the gradient descent, now with alpha as an optional argument"""
function gradient_descent(f, Df, x; alpha = 0.1)
    
    for N_iter in 1:100
        x = x - alpha*Df(x)
    end
    return (x, f(x))
end

println("Testing our implementation:")
println(gradient_descent(f, Df, 1))
# Change learning rate

println("\nTesting the learning rate:")
println(gradient_descent(f, Df, 1, alpha=0.01))

Testing our implementation:
(2.0370359763344877e-10, 4.1495155688809995e-20)

Testing the learning rate:
(0.13261955589475316, 0.017587946605721556)


As it was expected, diminishing the learning rate reduces the rate of convergence; in this case we are much farther from the minimum. Let's keep improving our `gradient_descent` in the following exercise:

#### Exercise 1:

Improve the `gradient_descent` functions by adding:

* *a tolerance and a maximum number of iterations*: stop the iterations only when `Df(x)` is smaller than the tolerance or the maximum number of iterations has been reached.
* *progress info*: print the current number of iterations, the value of `x` and that of `f(x)` every 100 iterations (*Hint*: in the second code cell you can see how to format variables into text) 
* *a `verbose` parameter*, to activate or deactivate the showing of progress info.

In [29]:
""" Third version of the gradient descent. Stops when the gradient is smaller than `TOL`, 
    or when the maximum number of iterations `maxiter` has been reached"""
function gradient_descent(f, Df, x; alpha = 0.1, TOL = 1e-10, maxiter = 1000, verbose = false)
    
    # Here goes your code
    
    return (x, f(x))
end

gradient_descent

In [28]:
# Let's test it:
gradient_descent(f, Df, 1, alpha = 0.01, maxiter = 10000, verbose = true)

Iter. 100,	x = 0.13261955589475316,	f(x) = 0.017587946605721556
Iter. 200,	x = 0.017587946605721556,	f(x) = 0.00030933586580571244
Iter. 300,	x = 0.002332505667951425,	f(x) = 5.440582691025523e-6
Iter. 400,	x = 0.00030933586580571244,	f(x) = 9.568867787376974e-8
Iter. 500,	x = 4.102398514547257e-5,	f(x) = 1.6829673572159537e-9
Iter. 600,	x = 5.4405826910255245e-6,	f(x) = 2.959994001788654e-11
Iter. 700,	x = 7.215276602924866e-7,	f(x) = 5.2060216456715e-13
Iter. 800,	x = 9.56886778737698e-8,	f(x) = 9.156323073230082e-15
Iter. 900,	x = 1.2690189963775444e-8,	f(x) = 1.61040921316707e-16
Iter. 1000,	x = 1.6829673572159529e-9,	f(x) = 2.8323791254544486e-18
Iter. 1100,	x = 2.2319438349934617e-10,	f(x) = 4.981573282565321e-20


(4.9049991751351605e-11, 2.4059016908076606e-21)

# Bonus: Multiple dispatch: things start getting different

Imagine for a moment that we don't want to care much about the initial point `x`. This may happen for example if we just want to get a local minimum of `f` and not the smallest one. Then it may make sense to define another `gradient descent` that just chooses `x` randomly. We could then make `x` also an optional argument; nevertheless we are going to explore another way that explotes one of Julia's most distinguishing features: *multiple dispatch*. To see how it works, let's consider this alternative version:

In [16]:
""" A fourth version of gradient descent, where the initial point gets initialized randomly.
    Stops when the gradient is smaller than `TOL`, or when the maximum number of iterations 
    `maxiter` has been reached"""
function gradient_descent(f, Df; alpha = 0.1, TOL = 1e-10, maxiter = 1000, verbose = false)
    x = rand()
    gradient_descent(f, Df, x; alpha = alpha, TOL = TOL, maxiter = maxiter, verbose = verbose)
end

# Let us run first the original version
x1,_ = gradient_descent(f, Df, 1, alpha = 0.1, maxiter = 10000);

# And then the new version
x2, _ = gradient_descent(f, Df, alpha = 0.1, maxiter = 10000);

println("The difference between the two solutions is $(abs(x1-x2)).")

The difference between the two solutions is 4.223326366518295e-12.


As we have seen above, both versions of `gradient_descent` coexist despite having used the same name. In fact, in Julia a function can have multiple *methods*, depending on which the inputs are. Thus, our `gradient_descent` has two methods:

In [17]:
methods(gradient_descent)

# 2 methods for generic function "gradient_descent":
[1] gradient_descent(f, Df; alpha, TOL, maxiter, verbose) in Main at In[16]:5
[2] gradient_descent(f, Df, x; alpha, TOL, maxiter, verbose) in Main at In[14]:5

Other functions, like the `sort` function, have a lot more methods defined:

In [3]:
methods(sort)

# 8 methods for generic function "sort":
[1] sort(r::AbstractUnitRange) in Base at range.jl:969
[2] sort(r::AbstractRange) in Base at range.jl:972
[3] sort(x::SparseArrays.SparseVector{Tv,Ti}; kws...) where {Tv, Ti} in SparseArrays at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/SparseArrays/src/sparsevector.jl:1913
[4] sort(v::AbstractArray{T,1} where T; kws...) in Base.Sort at sort.jl:751
[5] sort(A::AbstractArray; dims, alg, lt, by, rev, order) in Base.Sort at sort.jl:972
[6] sort(d::OrderedCollections.OrderedDict; args...) in OrderedCollections at /home/ismael/.julia/packages/OrderedCollections/E21Rb/src/dict_sorting.jl:20
[7] sort(d::Dict; args...) in OrderedCollections at /home/ismael/.julia/packages/OrderedCollections/E21Rb/src/dict_sorting.jl:21
[8] sort(d::OrderedCollections.LittleDict; byvalue, args...) in OrderedCollections at /home/ismael/.julia/packages/OrderedCollections/E21Rb/src/dict_sorting.jl:24

As you may guess from this list, the function `sort` has methods defined for a lot of relevant `types`: arrays, dict, ordered collections... And this is something we can also do by just annotating the type of the function's input. This adds even more flexibility: not only can we have different methods for different number of inputs, we can also have different methods for different *types* of the input, **and all behind the same function name**. Having different methods for different types helps Julia to compile the program in the most efficient form, which translates into improved performance.

On the other hand, this *multiple dispatch* also opens the door to reuse names of functions from standard libraries for user defined types if they perform a similar task. For example, Julia doesn't provide a method for sorting a tuple (and it may make sense, since tuples are supposed to be "immutable"):

In [4]:
A = (1, 3, 2)
sort(A) # this doesn't work

MethodError: MethodError: no method matching sort(::Tuple{Int64,Int64,Int64})
Closest candidates are:
  sort(!Matched::AbstractUnitRange) at range.jl:969
  sort(!Matched::AbstractRange) at range.jl:972
  sort(!Matched::SparseArrays.SparseVector{Tv,Ti}; kws...) where {Tv, Ti} at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/SparseArrays/src/sparsevector.jl:1913
  ...

However, we may as well write a method for sorting tuples that will get incorporated to the `sort` function methods.

In [20]:
import Base: sort                      # we must explicitly import the function to extend it
sort(T::Tuple) = Base.sort(collect(T)) # here ::Tuple signals that the method is supposed to work for tuples
methods(sort)

# 9 methods for generic function "sort":
[1] sort(r::AbstractUnitRange) in Base at range.jl:969
[2] sort(r::AbstractRange) in Base at range.jl:972
[3] sort(x::SparseArrays.SparseVector{Tv,Ti}; kws...) where {Tv, Ti} in SparseArrays at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/SparseArrays/src/sparsevector.jl:1913
[4] sort(v::AbstractArray{T,1} where T; kws...) in Base.Sort at sort.jl:751
[5] sort(A::AbstractArray; dims, alg, lt, by, rev, order) in Base.Sort at sort.jl:972
[6] sort(d::OrderedCollections.OrderedDict; args...) in OrderedCollections at /home/ismael/.julia/packages/OrderedCollections/E21Rb/src/dict_sorting.jl:20
[7] sort(d::Dict; args...) in OrderedCollections at /home/ismael/.julia/packages/OrderedCollections/E21Rb/src/dict_sorting.jl:21
[8] sort(d::OrderedCollections.LittleDict; byvalue, args...) in OrderedCollections at /home/ismael/.julia/packages/OrderedCollections/E21Rb/src/dict_sorting.jl:24
[9] sort(T::Tuple) in Main at In[20]:2

In [30]:
sort(A)

3-element Array{Int64,1}:
 1
 2
 3